In [1]:
import carball
import pandas as pd
import numpy as np
from google.protobuf.json_format import MessageToDict
from tqdm.notebook import tqdm

In [2]:
replay_name = '012194B14489DD6AD776F0A877C53C05.replay'
output_name = '012194B14489DD6AD776F0A877C53C05.json'

In [3]:
analysis_manager = carball.analyze_replay_file(replay_name, output_path=output_name, overwrite=True)

RattleTrapException: b'rattletrap-9.0.7-linux: 012194B14489DD6AD776F0A877C53C05.replay: openBinaryFile: does not exist (No such file or directory)'. Code: 1

In [ ]:
proto_game = analysis_manager.get_protobuf_data()

In [ ]:
df = analysis_manager.get_data_frame()

In [ ]:
dict_game = MessageToDict(proto_game)

In [ ]:
player_team = {}
best_score = 0
for i in dict_game['players']:
    # indentifies MVP
    if i['score'] > best_score:
        best_score = i['score']
    if i['isOrange']:
        player_team.update({i['name']: tuple([i['score'], 'orange'])})
    else:
        player_team.update({i['name']: tuple([i['score'], 'blue'])})
        
print("###########################\nTeams:")
for name, score in player_team.items():
    print(name + " (Scored " + str(score[0]) + ") " + ": " + score[1])
print('###########################\n')
print("*Best player*:")

ordered_playas = []
for name, score in player_team.items():
    if score[0] == best_score:
        ordered_playas.append(name)
        print(name + ": " + str(score[0]))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df.iloc[0])

In [ ]:
playas = list(list(df.columns.levels)[0])
playas.remove('ball')
playas.remove('game')
print(playas)

In [ ]:
for playa in playas:
    if playa not in ordered_playas and player_team[playa][1] == player_team[ordered_playas[0]][1]:
        ordered_playas.append(playa)

for playa in playas:
    if playa not in ordered_playas and player_team[playa][1] != player_team[ordered_playas[0]][1]:
        ordered_playas.append(playa)
        
print(ordered_playas)

In [ ]:
for playa in ordered_playas:
    print(df[playa].columns)
print(type(df[playa]))

In [ ]:
########################### MAKING SURE EACH LEVEL IS SAME LENGTH
length = len(df['ball'])
if length != len(df['game']):
    print("BAD")
for playa in ordered_playas:
    if len(df[playa]) != length:
        print("BAD")
###########################

In [ ]:
## MAKING NEW SINGLE LEVEL DF FOR TRAINING ##
## PLAYER 0, 1, AND 1 ARE HIGHEST SCORING PLAYER'S TEAM WHILE 3, 4, AND 5 ARE ON OPPOSITE TEAM ##
#first = True
player_desired = 0

print('player ' + str(player_desired) + ' is ' + ordered_playas[player_desired])
single_level_df = df[ordered_playas[player_desired]]
single_level_df.drop(columns=['rot_x','rot_y','rot_z','vel_x','vel_y','vel_z','ang_vel_x','ang_vel_y','ang_vel_z','ping','throttle','steer','handbrake','ball_cam','boost','boost_active','jump_active','double_jump_active','dodge_active','boost_collect'], inplace=True)
single_level_df.rename(columns={'pos_x': str(player_desired)+'_pos_x', 'pos_y': str(player_desired)+'_pos_y', 'pos_z': str(player_desired)+'_pos_z'}, inplace=True)
for i, playa in enumerate(ordered_playas):
    if player_desired == i:
        continue
    piece = df[playa]
    piece.drop(columns=['ping','throttle','steer','handbrake','ball_cam','boost','boost_active','jump_active','double_jump_active','dodge_active','boost_collect'], inplace=True)
    piece.rename(columns={'pos_x': str(i)+'_pos_x', 'pos_y': str(i)+'_pos_y', 'pos_z': str(i)+'_pos_z', 'rot_x': str(i)+'_rot_x', 'rot_y': str(i)+'_rot_y', 'rot_z': str(i)+'_rot_z', 'vel_x': str(i)+'_vel_x', 'vel_y': str(i)+'_vel_y', 'vel_z': str(i)+'_vel_z', 'ang_vel_x': str(i)+'_ang_vel_x', 'ang_vel_y': str(i)+'_ang_vel_y', 'ang_vel_z': str(i)+'_ang_vel_z'}, inplace=True)
    single_level_df = single_level_df.join(piece)
    print('player ' + str(i) + ' is ' + playa)
    
ball_data = df['ball']
ball_data.drop(columns=['hit_team_no'], inplace=True)
ball_data.rename(columns={'pos_x': 'ball_pos_x', 'pos_y': 'ball_pos_y', 'pos_z': 'ball_pos_z', 'rot_x': 'ball_rot_x', 'rot_y': 'ball_rot_y', 'rot_z': 'ball_rot_z', 'vel_x': 'ball_vel_x', 'vel_y': 'ball_vel_y', 'vel_z': 'ball_vel_z', 'ang_vel_x': 'ball_ang_vel_x', 'ang_vel_y': 'ball_ang_vel_y', 'ang_vel_z': 'ball_ang_vel_z'}, inplace=True)
single_level_df = single_level_df.join(ball_data)
single_level_df['seconds_remaining'] = df['game']['seconds_remaining']

In [ ]:
print(single_level_df.columns)
print(len(single_level_df))

In [ ]:
## CHECK HOW MANY NAN EXIST ## 
num_nans = len(single_level_df) - len(single_level_df.dropna())
print(str(num_nans), 'frames lost due to demolitions or players leaving')
single_level_df.dropna(inplace=True)
single_level_df.reset_index(drop=True, inplace=True)

In [ ]:
## SPLIT POSITIONING INTO SEGMENTS ##
## 8 X SEGMENTS, 10 Y SEGMENTS, 7 Z SEGMENTS
## THIS GIVES 1024 (X) BY 1024 (Y) BY 292 (Z) CUBES

def x_segment(x):
    seg = -4096
    while(True):
        if x >= seg and x < seg+1024:
            return (seg+(seg+1024))/2
        seg += 1024
        
def y_segment(y):
    seg = -5120
    if y < -5120:
        return -5121 # IN BLUE TEAM GOAL
    elif y > 5120:
        return 5121 # IN ORANGE TEAM GOAL
    while(True):
        if y >= seg and y < seg+1024:
            return (seg+(seg+1024))/2
        seg += 1024
        
def z_segment(z):
    seg = 0
    while(True):
        if z >= seg and z < seg+292:
            return (seg+(seg+292))/2
        seg += 292
        
length = len(single_level_df)
for i in tqdm(single_level_df.index):
    single_level_df.at[i, '0_pos_x'] = x_segment(single_level_df.at[i, '0_pos_x'])
    single_level_df.at[i, '0_pos_y'] = y_segment(single_level_df.at[i, '0_pos_y'])
    single_level_df.at[i, '0_pos_z'] = z_segment(single_level_df.at[i, '0_pos_z'])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(single_level_df.iloc[0])

In [ ]:
single_level_df.to_excel('all_objects.xlsx')

In [ ]:
## NEXT STEPS ARE TO FUNCTIONALIZE DF NORMALIZATION SO ALL REPLAYS CAN BE COMBINED INTO ONE DF ##